# BioExplorer - Neurons
![](../bioexplorer_neurons_banner.png)

### Connect to back-end

In [177]:
from bioexplorer import BioExplorer, NeuronDisplacementParams, Vector2

url = 'localhost:5000'
be = BioExplorer(url)

core = be.core_api()
status = be.reset_scene()

### Load neurons

In [178]:
core.set_geometry_parameters(
    sdf_nb_march_iterations=32, sdf_epsilon=0.0001,
    sdf_blend_factor=0.4, sdf_blend_lerp_factor=0.2,
    sdf_ray_marching_omega=1.0, sdf_distance=400.0
)

True

In [179]:
assembly_name = 'Neurons'
be.remove_assembly(assembly_name)
neurons_assembly = be.add_assembly(assembly_name)

neurons_model = be.add_neurons(
    assembly_name=assembly_name,
    population_name='o1',
    # morphology_representation=be.MORPHOLOGY_REPRESENTATION_SEGMENT,
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_BEZIER,
    # morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
    population_color_scheme=be.POPULATION_COLOR_SCHEME_ID,
    # radius_multiplier=0.8,
    realism_level=be.MORPHOLOGY_REALISM_LEVEL_ALL,
    # realism_level=be.MORPHOLOGY_REALISM_LEVEL_SOMA,
    synapses_type=be.NEURON_SYNAPSES_AFFERENT,
    generate_varicosities=False,
    # load_axon=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=False, generate_externals=False,
    # sql_node_filter='guid=49',
    sql_node_filter='guid%1000=0',
    displacement_params=NeuronDisplacementParams(
        soma=Vector2(0.1, 1.0), section=Vector2(0.1, 1.0),
        nucleus=Vector2(0.02, 1.0), mitochondrion=Vector2(0.4, 1.0),
        myelin_steath=Vector2(0.15, 4.0), spine=Vector2(0.01, 50.0))
)

In [180]:
# status = core.set_camera(
#     orientation=[-0.04516827475434093, -0.12648330315414616, 0.002897551396689663, 0.9909356211050424],
#     position=[419.9380089738668, 1864.926535721847, 215.43552754741884],
#     target=[452.2792946677419, 1853.483701015516, 91.20223249410772]
# )
# status = core.set_renderer()

In [181]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name, palette_size):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, palette_size)

    for material_id in material_ids:
        mid = material_id % palette_size
        colors.append(palette[mid])
        if mid in [be.NEURON_MATERIAL_AFFERENT_SYNAPSE, be.NEURON_MATERIAL_EFFERENT_SYNAPSE]:
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.001)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MITOCHONDRION:
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.5)
            user_params.append(1.0)
            specular_exponents.append(6.0)
            emissions.append(0.25)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(1.0)
            user_params.append(0.2)
            specular_exponents.append(30.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            opacities.append(0.75)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.2)
            user_params.append(0.01)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_SHEATH:
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_DIFFUSE)
            glossinesses.append(1.0)
            user_params.append(1.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        else:
            # Membrane
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.07)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            
        refraction_indices.append(1.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [184]:
model_ids = be.get_model_ids()['ids']
offset = model_ids[0]
set_morphology_materials(offset, 'Set3', be.NB_MATERIALS_PER_MORPHOLOGY)
core.set_renderer()

True

In [183]:
status = core.set_renderer(
    background_color=[1,1,1],
    current='advanced',subsampling=4, max_accum_frames=64)
params = core.AdvancedRendererParams()
params.gi_ray_length = 100.0
params.shadow_intensity = 0.5
params.soft_shadow_strength = 0.0
params.epsilon_multiplier = 100.0
params.max_ray_depth = 1
params.show_background = False
params.main_exposure = 1.1
status = core.set_renderer_params(params)
status = core.set_renderer()

In [185]:
from bioexplorer import Widgets
w = Widgets(be)

In [186]:
w.display_palette_for_models()